In [1]:
import os
import requests
from dotenv import load_dotenv, find_dotenv
from langchain_openai import OpenAI, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType, load_tools

In [2]:
# Load environment API Keys variables from .env file 
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')

In [3]:
# Define the prompt templates
condense_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""
Given the conversation history and the latest user query, rephrase the question to be standalone, keeping it concise but maintaining all necessary details.

### Conversation History:
{chat_history}

### Latest User Query:
{question}

### Standalone Question for Retrieval:
""")

combine_docs_custom_prompt = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
You are an AI travel planner helping users design an itinerary. Use the retrieved information about landmarks and the user's past preferences to generate a relevant and coherent travel recommendation.

### Conversation History:
{chat_history}

### User's Latest Question:
{question}

### Retrieved Landmark Information:
{context}

### Instructions:
- Provide a well-structured travel recommendation based on the retrieved landmarks.
- Ensure continuity with previous discussions.
- Prioritize landmarks that match the user’s preferences.
- If multiple options exist, suggest the best ones with reasoning.
- Avoid repeating information already given in the conversation.
- In the end ask the user which of these locations you will like to visit.

### Final Answer:
""")

In [4]:
#Initialize LLM with ReAct Chain
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model="gpt-4-turbo")

# Define memory to store conversation history
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

# Function to print chat history
def print_chat_history():
    print("\nChat History:")
    for idx, msg in enumerate(memory.chat_memory.messages):
        role = "User" if msg.type == "human" else "AI"
        print(f"{role}: {msg.content}")

# Connect to your existing ChromaDB collection
vectorstore = Chroma(
    collection_name="landmarks_rag",
    embedding_function=OpenAIEmbeddings()
)

#Define the retriever and chain
retriever = vectorstore.as_retriever(k=3)  # This fetches relevant landmarks

# Set up the conversational retrieval chain
retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Using OpenAI's ChatGPT as the LLM
    retriever=retriever,  # Connect to your ChromaDB retriever
    memory=memory,
    condense_question_prompt=condense_prompt,  # Ensures refined queries for retrieval
    combine_docs_chain_kwargs=dict(prompt=combine_docs_custom_prompt)  # Customizes how retrieved docs are used
)

# Wrap QA Chain as a Tool 
qa_tool = Tool( name="Puerto Rico Travel Guide",
               func=retrieval_chain.run,
               description="Retrieve the best places to visit in Puerto Rico based on user queries." 
               )

C:\Users\larry\AppData\Local\Temp\ipykernel_1932\3204885827.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [5]:
# =======================# 2. Extracting Locations from QA Response# ======================= 
location_extraction_prompt = PromptTemplate( input_variables=["response"], 
                                            template=""" Extract only the location names from the following text: "{response}" Provide the locations as a comma-separated list. """ 
                                            ) 
                                            
location_extraction_chain = LLMChain( llm=llm,
                                     prompt=location_extraction_prompt )

def extract_locations_from_response(response):
    """Extracts locations using the LLM chain."""
    location_list = location_extraction_chain.run(response)
    return [loc.strip() for loc in location_list.split(",") if loc.strip()] 

# =======================# 3. Asking the User for Their Selected Places# =======================
def ask_user_for_places(query): 
    """Asks the user to select places they are interested in visiting."""
    # Retrieve recommended locations from QA #
    recommended_places = qa_tool.run(query) 
    extracted_locations = extract_locations_from_response(recommended_places)
    if extracted_locations:
        weather_details = get_weather_for_selected_places(extracted_locations)
        res = f"Here are some great places to visit: {', '.join(extracted_locations)}. {weather_details}. Which ones do you want to visit?"
        return res
    else:
        return "I couldn't find relevant locations. Please try another query."
    # Wrap as a Tool

ask_places_tool = Tool( name="Ask User for Selected Places", func=ask_user_for_places, description="Ask the user which places they want to visit from the recommended list." )

C:\Users\larry\AppData\Local\Temp\ipykernel_1932\3024074906.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  location_extraction_chain = LLMChain( llm=llm,


In [ ]:
# =======================# 4. Getting Weather for Selected Locations# =======================# 
def get_weather(location):
    """Fetch real-time weather for a given location."""
    api_key = OPENWEATHER_API_KEY  # Replace with your API key
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": location, "appid": api_key, "units": "metric"}
    response = requests.get(base_url, params=params) 
    
    if response.status_code == 200:
        data = response.json()
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {location} is {weather} with a temperature of {temp}°C."
    
    else: return f"Could not fetch weather data for {location}."
    
def get_weather_for_selected_places(selected_places):
    """Fetches weather for the user-selected locations."""
    locations = [loc.strip() for loc in selected_places.split(",") if loc.strip()]
    if not locations:
        return "Please provide at least one valid location."
    weather_reports = [get_weather(location) for location in locations]
    return "\n".join(weather_reports) 

# Wrap Weather Fetching as a Tool
weather_tool = Tool( name="Get Weather for Selected Places", func=get_weather_for_selected_places, description="Retrieve weather for the places selected by the user." )

In [ ]:
agent = initialize_agent( tools=[qa_tool, ask_places_tool, weather_tool],
                          # Adding all tools 
                          llm=llm,
                          agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, # Keeps conversation context
                          verbose=True,
                          memory=memory 
                          )

In [ ]:
# Step 1: Get recommended places
response1 = agent.run("What are the best places to visit in Puerto Rico?")

print(response1)


In [ ]:
# Expected: "Here are some great places: San Juan, El Yunque, Culebra... Which ones do you want to visit?"# Simulate User Input 
user_selected_places = "San Juan and Ponce"

# Step 2: Fetch weather for selected places 
response2 = agent.run(f"I want to visit {user_selected_places}")

print(response2) # Expected: Weather details for "San Juan" and "El Yunque".

In [ ]:

# Step 2: Fetch weather for selected places 
response2 = agent.run("what is the wether in those places?")

print(response2) # Expected: Weather details for "San Juan" and "El Yunque".

In [ ]:
import os
import requests
from dotenv import load_dotenv, find_dotenv
from langchain_openai import OpenAI, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

# Load API Keys
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')

# Municipality Mapping
municipality_mapping = {
    'Adjuntas': 'Adjuntas, PR', 'Aguada': 'Aguada, PR', 'Aguadilla': 'Aguadilla, PR', 'Aguas Buenas': 'Aguas Buenas, PR', 'Aibonito': 'Aibonito, PR',
     'Arecibo': 'Arecibo, PR', 'Arroyo': 'Arroyo, PR', 'Añasco': 'Añasco, PR', 'Barceloneta': 'Barceloneta, PR', 'Barranquitas': 'Barranquitas, PR',
     'Bayamón': 'Bayamón, PR', 'Cabo Rojo': 'Cabo Rojo, PR', 'Caguas': 'Caguas, PR', 'Camuy': 'Camuy, PR', 'Canóvanas': 'Canóvanas, PR',
     'Carolina': 'Carolina, PR', 'Cataño': 'Cataño, PR', 'Cayey': 'Cayey, PR', 'Ceiba': 'Ceiba, PR', 'Ciales': 'Ciales, PR', 'Cidra': 'Cidra, PR',
     'Coamo': 'Coamo, PR', 'Comerío': 'Comerío, PR', 'Corozal': 'Corozal, PR', 'Culebra': 'Culebra, PR', 'Dorado': 'Dorado, PR', 'Fajardo': 'Fajardo, PR',
     'Florida': 'Florida, PR', 'Guayama': 'Guayama, PR', 'Guayanilla': 'Guayanilla, PR', 'Guaynabo': 'Guaynabo, PR', 'Gurabo': 'Gurabo, PR',
     'Guánica': 'Guánica, PR', 'Hatillo': 'Hatillo, PR', 'Hormigueros': 'Hormigueros, PR', 'Humacao': 'Humacao, PR', 'Isabela': 'Isabela, PR',
     'Jayuya': 'Jayuya, PR', 'Juana Díaz': 'Juana Díaz, PR', 'Juncos': 'Juncos, PR', 'Lajas': 'Lajas, PR', 'Lares': 'Lares, PR',
     'Las Marías': 'Las Marías, PR', 'Las Piedras': 'Las Piedras, PR', 'Loíza': 'Loíza, PR', 'Luquillo': 'Luquillo, PR', 'Manatí': 'Manatí, PR',
     'Maricao': 'Maricao, PR', 'Maunabo': 'Maunabo, PR', 'Mayagüez': 'Mayagüez, PR', 'Moca': 'Moca, PR', 'Morovis': 'Morovis, PR',
     'Naguabo': 'Naguabo, PR', 'Naranjito': 'Naranjito, PR', 'Orocovis': 'Orocovis, PR', 'Patillas': 'Patillas, PR', 'Peñuelas': 'Peñuelas, PR',
     'Ponce': 'Ponce, PR', 'Quebradillas': 'Quebradillas, PR', 'Rincón': 'Rincón, PR', 'Río Grande': 'Río Grande, PR',
     'Sabana Grande': 'Sabana Grande, PR', 'Salinas': 'Salinas, PR', 'San Germán': 'San Germán, PR', 'San Juan': 'San Juan, PR',
     'San Lorenzo': 'San Lorenzo, PR', 'San Sebastián': 'San Sebastián, PR', 'Santa Isabel': 'Santa Isabel, PR', 'Toa Alta': 'Toa Alta, PR',
     'Toa Baja': 'Toa Baja, PR', 'Trujillo Alto': 'Trujillo Alto, PR', 'Utuado': 'Utuado, PR', 'Vega Alta': 'Vega Alta, PR', 'Vega Baja': 'Vega Baja, PR',
     'Vieques': 'Vieques, PR', 'Villalba': 'Villalba, PR', 'Yabucoa': 'Yabucoa, PR', 'Yauco': 'Yauco, PR'
}

# ========== 1. Prompt Templates ==========
condense_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""
Given the conversation history and the latest user query, rephrase the question to be standalone.

### Conversation History:
{chat_history}

### User Query:
{question}

### Rephrased Question:
"""
)

combine_docs_custom_prompt = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
You are an AI travel planner helping users design an itinerary.

### Conversation History:
{chat_history}

### User's Question:
{question}

### Retrieved Landmark Data:
{context}

### Instructions:
- Provide a well-structured day-by-day itinerary.
- Ensure continuity with previous discussions.
- Prioritize locations based on user interest.
- If multiple options exist, suggest the best ones.
- Avoid repeating information already given.
- At the end, ask the user how many days they are staying.

### Final Itinerary:
"""
)

# ========== 2. Initialize LLM and Memory ==========
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model="gpt-4-turbo")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ========== 3. ChromaDB Retrieval ==========
vectorstore = Chroma(collection_name="landmarks_rag", embedding_function=OpenAIEmbeddings(api_key=OPENAI_API_KEY))
retriever = vectorstore.as_retriever(k=3)

retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    chain_type="stuff",
    condense_question_prompt=condense_prompt,
    combine_docs_chain_kwargs=dict(prompt=combine_docs_custom_prompt)
)

qa_tool = Tool(
    name="Puerto Rico Travel Guide",
    func=retrieval_chain.invoke,
    description="Retrieve top locations to visit in Puerto Rico based on user preferences."
)

# ========== 4. Extract Locations ==========
location_extraction_prompt = PromptTemplate(
    input_variables=["response"],
    template="Extract only the names of locations from this text: {response}. Provide them as a comma-separated list."
)

location_extraction_chain = LLMChain(llm=llm, prompt=location_extraction_prompt)

def extract_locations_from_response(response):
    """Extracts locations using LLM processing."""
    location_list = location_extraction_chain.invoke({"response": response})  # FIXED: Correct input format
    extracted_places = [loc.strip() for loc in location_list.split(",") if loc.strip()]
    valid_locations = [loc for loc in extracted_places if loc in municipality_mapping]
    return valid_locations

# ========== 5. Extract Number of Days from Memory ==========
def get_trip_duration():
    """Infers the number of days from conversation memory."""
    messages = memory.chat_memory.messages
    for message in reversed(messages):
        if "days" in message.content.lower():
            try:
                words = message.content.split()
                for i, word in enumerate(words):
                    if word.isdigit():  # If a number is found
                        return int(word)
            except ValueError:
                continue
    return None  # If not found

# ========== 6. Itinerary Generator ==========
import random

def generate_itinerary(selected_places):
    """Generates a structured itinerary by distributing locations across inferred days."""
    num_days = get_trip_duration()
    
    if num_days is None:
        num_days = int(input("\nHow many days is your trip? "))  # Ask only if missing
        memory.chat_memory.add_user_message(f"My trip is {num_days} days.")  # Store in memory

    itinerary = ""
    places_per_day = max(1, len(selected_places) // num_days)
    random.shuffle(selected_places)

    for day in range(1, num_days + 1):
        start_idx = (day - 1) * places_per_day
        end_idx = start_idx + places_per_day
        day_places = selected_places[start_idx:end_idx]

        if not day_places:
            break

        itinerary += f"\n📅 **Day {day}:**\n"
        for place in day_places:
            itinerary += f"- Visit **{place}** (Located in {municipality_mapping.get(place, 'Unknown Municipality')})\n"

    return itinerary.strip()

# ========== 7. Weather Retrieval ==========
def get_weather(location):
    """Fetch real-time weather for a given municipality."""
    formatted_location = municipality_mapping.get(location, location)

    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": formatted_location, "appid": OPENWEATHER_API_KEY, "units": "metric"}

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {formatted_location} is {weather} with a temperature of {temp}°C."
    else:
        return f"Could not fetch weather data for {formatted_location}."

def get_weather_for_selected_places(selected_places):
    """Fetches weather for selected locations."""
    locations = [loc.strip() for loc in selected_places if loc.strip()]
    if not locations:
        return "Please provide at least one valid location."
    
    weather_reports = [get_weather(location) for location in locations]
    return "\n".join(weather_reports)

weather_tool = Tool(
    name="Get Weather for Selected Places",
    func=get_weather_for_selected_places,
    description="Retrieve weather for selected travel locations."
)

# ========== 8. AI Agent ==========
agent = initialize_agent(
    tools=[qa_tool, weather_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

# ========== 9. Simplified Chatbot Execution ==========
def travel2pr_planner():
    print("\n🚀 Welcome to the AI Travel Planner!")
    print("💡 Type 'exit' anytime to stop.\n")

    while True:
        user_query = input("\nUser: ")
        if user_query.lower() == "exit":
            print("\n👋 Exiting AI Travel Planner. Safe travels!")
            break

        response = agent.invoke(user_query)  # FIXED: Correct method for agent execution
        print("\n📝 AI Response:\n", response)

        extracted_locations = extract_locations_from_response(response)

        if not extracted_locations:
            print("\n⚠️ No valid locations found. Let's refine the search...")
            continue  # Ask again

        itinerary = generate_itinerary(extracted_locations)
        weather_info = get_weather_for_selected_places(extracted_locations)

        print("\n📅 **Your Itinerary:**\n", itinerary)
        print("\n🌤 **Weather Report:**\n", weather_info)

In [ ]:
# Run the chatbot
travel2pr_planner()